In [1]:
"""
 This code scrap raw data from tweeter using twint library , convert it into csv file then perform preprocessing

"""

import twint
import nest_asyncio
import pandas as pd
import config
import matplotlib.pyplot as plt
import rubrix as rb
from transformers import pipeline
import datasets
import numpy as np


In [2]:
search= ['COVID-19 vaccine']
temp_name = search[0].replace(" ","_")+".csv"

## Scraping with twint

In [27]:
# Run scraping
nest_asyncio.apply()
c = twint.Config()
all_topics = []
search= ['COVID-19 vaccine']
c.Search = [search]       # topic
c.Limit = config.TWEETS_TO_SCRAP      # number of Tweets to scrape
c.Store_csv = True       # store tweets in a csv file
temp_name = search.replace(" ","_")+".csv"
c.Output = temp_name    # path to csv file
c.Lang = "en"
twint.run.Search(c)

1557051089349910530 2022-08-09 19:08:02 +0200 <Indus_Helps> Do you need your first, second, or booster COVID-19 vaccine, or even your second booster? Then attend the GO-VAXX bus COVID-19 clinics for doses available for eligible individuals age 5 &amp; over.  #IndusHelps #Vaccines  https://t.co/zmKbqFIehi
1557050583332315137 2022-08-09 19:06:02 +0200 <JLionlight> @thevivafrei Most unvaccinated have noticed oddities in the approach the government took to the Covid 19 vaccine and it seems more and more we had good reason.   You should do a video highlighting all the oddities since the start. The price of gas dropped to 68 cents a liter in Canada.
1557050279270486016 2022-08-09 19:04:49 +0200 <123_INFO_CA> COVID-19 vaccine mandate for Canadian military will be ‘tweaked,’ says defence chief #123INFO  https://t.co/LjC9yYvMoJ
1557049929100627971 2022-08-09 19:03:26 +0200 <RwandainSudan> #RwoT Rwanda rolls out fourth COVID-19 vaccine jabs   https://t.co/rszBxV2DXN via @cgtnofficial
15570494765

## Data cleanning and preprocessing

In [115]:
df = pd.read_csv(temp_name)

In [116]:
df.head()

,id,conversation_id,created_at,date,time,timezone,user_id,username,name,place,...,geo,source,user_rt_id,user_rt,retweet_id,reply_to,retweet_date,translate,trans_src,trans_dest
0,1557051089349910530,1557051089349910530,2022-08-09 19:08:02 CEST,2022-08-09,19:08:02,200,834865924045893632,indus_helps,Indus Community Services,NaN,...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
1,1557050583332315137,1556986431700180993,2022-08-09 19:06:02 CEST,2022-08-09,19:06:02,200,1457825105560616961,jlionlight,Jake Lionlight,NaN,...,NaN,NaN,NaN,NaN,NaN,"[{'screen_name': 'thevivafrei', 'name': 'Viva ...",NaN,NaN,NaN,NaN
2,1557050279270486016,1557050279270486016,2022-08-09 19:04:49 CEST,2022-08-09,19:04:49,200,1473622511363543041,123_info_ca,123 INFO CANADA,NaN,...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
3,1557049929100627971,1557049929100627971,2022-08-09 19:03:26 CEST,2022-08-09,19:03:26,200,1308353972,rwandainsudan,Rwanda in Sudan,NaN,...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
4,1557049476522692609,1557049476522692609,2022-08-09 19:01:38 CEST,2022-08-09,19:01:38,200,2372240520,tcrhcc,Tuba City Regional Health Care Corp.,NaN,...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN


In [30]:
df.columns # colonnes du dataset

Index(['id', 'conversation_id', 'created_at', 'date', 'time', 'timezone',
       'user_id', 'username', 'name', 'place', 'tweet', 'language', 'mentions',
       'urls', 'photos', 'replies_count', 'retweets_count', 'likes_count',
       'hashtags', 'cashtags', 'link', 'retweet', 'quote_url', 'video',
       'thumbnail', 'near', 'geo', 'source', 'user_rt_id', 'user_rt',
       'retweet_id', 'reply_to', 'retweet_date', 'translate', 'trans_src',
       'trans_dest'],
      dtype='object')

In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10014 entries, 0 to 10013
Data columns (total 36 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id               10014 non-null  int64  
 1   conversation_id  10014 non-null  int64  
 2   created_at       10014 non-null  object 
 3   date             10014 non-null  object 
 4   time             10014 non-null  object 
 5   timezone         10014 non-null  int64  
 6   user_id          10014 non-null  int64  
 7   username         10014 non-null  object 
 8   name             10014 non-null  object 
 9   place            7 non-null      object 
 10  tweet            10014 non-null  object 
 11  language         10014 non-null  object 
 12  mentions         10014 non-null  object 
 13  urls             10014 non-null  object 
 14  photos           10014 non-null  object 
 15  replies_count    10014 non-null  int64  
 16  retweets_count   10014 non-null  int64  
 17  likes_count 

In [117]:
# checking for missing values
df.isnull().sum()

id                     0
conversation_id        0
created_at             0
date                   0
time                   0
timezone               0
user_id                0
username               0
name                   0
place              10007
tweet                  0
language               0
mentions               0
urls                   0
photos                 0
replies_count          0
retweets_count         0
likes_count            0
hashtags               0
cashtags               0
link                   0
retweet                0
quote_url           9554
video                  0
thumbnail           7619
near               10014
geo                10014
source             10014
user_rt_id         10014
user_rt            10014
retweet_id         10014
reply_to               0
retweet_date       10014
translate          10014
trans_src          10014
trans_dest         10014
dtype: int64

In [118]:
df = df[['id','created_at','tweet','user_id']]
# dropping null rows
df.dropna(subset=['tweet'])
df.isnull().sum()

id            0
created_at    0
tweet         0
user_id       0
dtype: int64

In [120]:
df.info

<bound method DataFrame.info of                         id                created_at  \
0      1557051089349910530  2022-08-09 19:08:02 CEST   
1      1557050583332315137  2022-08-09 19:06:02 CEST   
2      1557050279270486016  2022-08-09 19:04:49 CEST   
3      1557049929100627971  2022-08-09 19:03:26 CEST   
4      1557049476522692609  2022-08-09 19:01:38 CEST   
...                    ...                       ...   
10009  1554878153981575169  2022-08-03 19:13:34 CEST   
10010  1554877902176526336  2022-08-03 19:12:34 CEST   
10011  1554877682092896256  2022-08-03 19:11:42 CEST   
10012  1554877657690640384  2022-08-03 19:11:36 CEST   
10013  1554877522096992257  2022-08-03 19:11:04 CEST   

                                                   tweet              user_id  
0      Do you need your first, second, or booster COV...   834865924045893632  
1      @thevivafrei Most unvaccinated have noticed od...  1457825105560616961  
2      COVID-19 vaccine mandate for Canadian military..

## Preprocessing and data labelling

In [121]:
def preprocess(text):
    new_text = []
    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)

df['tweet_prep'] = df['tweet'].apply(lambda x:preprocess(x))

In [122]:
print("Original tweet : {}".format(df['tweet'][0]))
print("Preprocessed tweet : {}".format(df['tweet_prep'][0]))

Original tweet : Do you need your first, second, or booster COVID-19 vaccine, or even your second booster? Then attend the GO-VAXX bus COVID-19 clinics for doses available for eligible individuals age 5 &amp; over.  #IndusHelps #Vaccines  https://t.co/zmKbqFIehi
Preprocessed tweet : Do you need your first, second, or booster COVID-19 vaccine, or even your second booster? Then attend the GO-VAXX bus COVID-19 clinics for doses available for eligible individuals age 5 &amp; over.  #IndusHelps #Vaccines  http


In [123]:
df.drop(columns=['tweet'])

,id,created_at,user_id,tweet_prep
0,1557051089349910530,2022-08-09 19:08:02 CEST,834865924045893632,"Do you need your first, second, or booster COV..."
1,1557050583332315137,2022-08-09 19:06:02 CEST,1457825105560616961,@user Most unvaccinated have noticed oddities ...
2,1557050279270486016,2022-08-09 19:04:49 CEST,1473622511363543041,COVID-19 vaccine mandate for Canadian military...
3,1557049929100627971,2022-08-09 19:03:26 CEST,1308353972,#RwoT Rwanda rolls out fourth COVID-19 vaccine...
4,1557049476522692609,2022-08-09 19:01:38 CEST,2372240520,Children 6 months and older are eligible for C...
...,...,...,...,...
10009,1554878153981575169,2022-08-03 19:13:34 CEST,115085178,A nasal vaccine could be key in curbing the sp...
10010,1554877902176526336,2022-08-03 19:12:34 CEST,1214209408751685632,@user @user The blood clotting is undeniable t...
10011,1554877682092896256,2022-08-03 19:11:42 CEST,795650829285896192,I agree with you. I even tweeted that it was a...
10012,1554877657690640384,2022-08-03 19:11:36 CEST,1547980003568017409,"Biden tests positive for COVID-19 again, will ..."


In [124]:
df.dropna(subset=['tweet_prep'])
df.isnull().sum()

id            0
created_at    0
tweet         0
user_id       0
tweet_prep    0
dtype: int64

In [125]:
df.info

<bound method DataFrame.info of                         id                created_at  \
0      1557051089349910530  2022-08-09 19:08:02 CEST   
1      1557050583332315137  2022-08-09 19:06:02 CEST   
2      1557050279270486016  2022-08-09 19:04:49 CEST   
3      1557049929100627971  2022-08-09 19:03:26 CEST   
4      1557049476522692609  2022-08-09 19:01:38 CEST   
...                    ...                       ...   
10009  1554878153981575169  2022-08-03 19:13:34 CEST   
10010  1554877902176526336  2022-08-03 19:12:34 CEST   
10011  1554877682092896256  2022-08-03 19:11:42 CEST   
10012  1554877657690640384  2022-08-03 19:11:36 CEST   
10013  1554877522096992257  2022-08-03 19:11:04 CEST   

                                                   tweet              user_id  \
0      Do you need your first, second, or booster COV...   834865924045893632   
1      @thevivafrei Most unvaccinated have noticed od...  1457825105560616961   
2      COVID-19 vaccine mandate for Canadian militar

In [127]:
df.to_csv('clean_'+temp_name)

In [79]:
## Data annotation with rubrix

# we use a zero-shot classifier
model = pipeline('zero-shot-classification', model="typeform/distilbert-base-uncased-mnli")

The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.


In [56]:
df = pd.read_csv("clean_COVID-19_vaccine.csv", header=0, engine='python',usecols=['id','user_id','tweet_prep','created_at'])
labels = [0,1,2]

In [128]:
df.describe()

,id,user_id
count,1.001400e+04,1.001400e+04
mean,1.555920e+18,6.366248e+17
std,6.539921e+14,6.614967e+17
min,1.554878e+18,1.031200e+04
25%,1.555324e+18,2.945110e+08
50%,1.555837e+18,4.872500e+09
75%,1.556583e+18,1.349061e+18
max,1.557051e+18,1.556885e+18


## Dataset labeling with rubrix 

In [107]:

records = []
for pos,item in df.iterrows():
    if item['tweet_prep'] is not None :
        prediction = model(item['tweet_prep'], labels)

        records.append(
            rb.TextClassificationRecord(
                text=item["tweet_prep"],
                prediction=list(zip(prediction['labels'], prediction['scores']))
            )
        )

rb.log(records, name="covid_vaccine")

100%|██████████| 2018/2018 [00:08<00:00, 235.04it/s]

2018 records logged to http://localhost:6900/datasets/rubrix/covid_vaccine


BulkResponse(dataset='covid_vaccine', processed=2018, failed=0)

In [29]:
# load the Rubrix dataset, perform hand labeling if necessary and saving it
#  into a pandas DataFrame after validation
rb_df = rb.load(name='covid_vaccine').to_pandas()

In [48]:
validated = rb_df[rb_df['status']=='Validated'][['text','annotation']]
validated = validated.rename(columns={"annotation": "labels"})

In [61]:
default = rb_df[rb_df['status']=='Default'][['text','prediction']]
default['labels'] = default['prediction'].map(lambda x:x[0][0])


In [62]:
labelled_df = pd.concat([validated,default])
labelled_df = labelled_df[['text','labels']]

In [65]:
labelled_df.to_csv('labelled_'+temp_name,columns=['text','labels'])

In [31]:
"""
from transformers import pipeline
model_path = f"cardiffnlp/twitter-roberta-base-sentiment-latest"
sentiment_task = pipeline("sentiment-analysis", model=model_path, tokenizer=model_path)
sentiment_task("Covid cases are increasing fast!")


labels_dict={
    'Negative':0,
    'Neutral':1,
    'Positive':2
}

#sentiment_task(list(df['tweet_prep'])[:3])
df['labels'] = df['tweet_prep'].apply(lambda x:labels_dict[sentiment_task(x)[0]['label']])
df['labels'][:10]
"""